In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import block_diag
from datetime import datetime as dt
import sympy as sym
from itertools import combinations
import math
from scipy.special import binom

In [9]:
tens = np.random.random((50, 14400, 609))

In [16]:
tens = tf.random.uniform((10, 10, 5))
tf.gather_nd(tens, tf.constant([[1, 2], [3, 4]]))

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0.5202941 , 0.17975104, 0.27964318, 0.16578794, 0.06142867],
       [0.6914742 , 0.3106817 , 0.9801209 , 0.8392924 , 0.20858705]],
      dtype=float32)>

In [19]:
tf.argsort(tens)[:, :, -2:]

<tf.Tensor: shape=(10, 10, 2), dtype=int32, numpy=
array([[[3, 0],
        [3, 1],
        [1, 4],
        [4, 1],
        [2, 4],
        [2, 4],
        [3, 1],
        [2, 4],
        [4, 1],
        [1, 4]],

       [[4, 1],
        [2, 4],
        [2, 0],
        [0, 2],
        [2, 3],
        [0, 1],
        [4, 1],
        [3, 2],
        [3, 0],
        [4, 1]],

       [[1, 4],
        [4, 3],
        [0, 4],
        [2, 4],
        [0, 4],
        [2, 0],
        [0, 3],
        [2, 4],
        [4, 2],
        [3, 1]],

       [[4, 3],
        [1, 4],
        [2, 0],
        [3, 0],
        [3, 2],
        [0, 1],
        [0, 3],
        [0, 2],
        [0, 4],
        [3, 0]],

       [[2, 3],
        [1, 0],
        [1, 4],
        [3, 0],
        [4, 1],
        [1, 3],
        [1, 0],
        [2, 3],
        [1, 3],
        [1, 3]],

       [[2, 4],
        [1, 0],
        [3, 1],
        [0, 4],
        [3, 0],
        [4, 1],
        [2, 3],
        [1, 0],
        [3,

In [ ]:
def calc_amplitude(x_vec_neighbours_idx, mo_features, gram, norb, cgc):
    sp_a_idx = x_vec_neighbours_idx[:, :norb]
    sp_b_idx = x_vec_neighbours_idx[:, norb:]
    ampl_a = tf.gather(mo_features, sp_a_idx)
    ampl_a_sq = tf.einsum("niaf,nibf,acb->nicf", ampl_a, ampl_a, cgc)
    ampl_b = tf.gather(mo_features, sp_b_idx)
    ampl_b_sq = tf.einsum("niaf,nibf,acb->nicf", ampl_b, ampl_b, cgc)
    ampl_a = tf.linalg.det("niaf,njaf,nij->nij", ampl_a, ampl_a_sq, gram)
    ampl_b = tf.linalg.det("niaf,njaf,nij->nij", ampl_b, ampl_b_sq, gram)
    ampl = ampl_a * ampl_b
    return ampl


def e_local(N_u, x_vec_as_dec, x_vec_as_idx, dets_as_idx, dets_as_dec, neighbour_dets_as_dec, neighbour_dets_matrix_el, mo_features, gram):
    # x_vec_as_dec (= x_vec): (N_u,); x_vec_as_idx: (N_u, nelec); dets_as_idx: (no_dets, nelec)
    # neighbour_dets_as_dec: (no_dets, no_neighbour_dets); neighbour_dets_matrix_el: (no_dets, no_neighbour_dets)
    norb = len(dets_as_idx[0])
    x_vec_neighbours_dec = tf.gather(neighbour_dets_as_dec, x_vec_as_dec)
    x_vec_neighbours_idx = tf.gather(dets_as_idx, x_vec_neighbours_dec)
    ampls = calc_amplitude(x_vec_neighbours_idx, mo_features, gram, norb)
    # ampls of shape (V_n, no_neighbour_dets)
    matr_els = tf.gather(neighbour_dets_matrix_el, x_vec_as_idx)
    denom = calc_amplitude(x_vec_as_idx, mo_features, gram, norb)
    e_loc = tf.reduce_sum(ampls * matr_els, axis=-1) / denom
    top_N_u_ampls_idx = tf.argsort(tf.reshape(ampls, [tf.shape(ampls)[0], tf.shape(ampls)[1] * tf.shape(ampls)[2]]))[:, -N_u:]
    top_N_u_dets_as_idx = tf.gather(dets_as_idx, top_N_u_ampls_idx)
    top_N_u_dets_as_dec = tf.gather(dets_as_dec, top_N_u_ampls_idx)
    return e_loc, denom, top_N_u_dets_as_idx, top_N_u_dets_as_dec

In [20]:
tf.argsort(tf.range(10))

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>